## Import libraries

In [16]:
import pandas as pd
import numpy as np

import ast

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score, roc_curve

from sklearn.linear_model import LogisticRegression

import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (16, 12)

In [17]:
import emoji
print(emoji.demojize('Python is 😘😍💞😙🌸😚♥️'))

Python is :face_blowing_a_kiss::smiling_face_with_heart-eyes::revolving_hearts::kissing_face_with_smiling_eyes::cherry_blossom::kissing_face_with_closed_eyes::heart_suit:


## Raw data reading

In [18]:
df = pd.read_csv('dataset/facebook_yessenov_2.csv')
df = df.loc[df.comments.dropna().index]
df['comments_dict'] = df.comments.apply(ast.literal_eval)
df

,id,created_time,message,story,comments,comments_dict
1,1849247698620787_2868153750063505,2021-04-13T15:51:44+0000,📌13 сәуір күні Yessenov University ұйымдастыру...,Yessenov University сейчас здесь: Yessenov Uni...,{'data': [{'created_time': '2021-04-13T18:52:1...,{'data': [{'created_time': '2021-04-13T18:52:1...
2,1849247698620787_2868147476730799,2021-04-13T15:39:30+0000,📌Yessenov University Білім беру мектебінің ұйы...,Yessenov University сейчас здесь: Yessenov Uni...,{'data': [{'created_time': '2021-04-13T19:55:3...,{'data': [{'created_time': '2021-04-13T19:55:3...
3,1849247698620787_2867867713425442,2021-04-13T06:38:10+0000,Yessenov University барлықтарыңызды қасиетті Р...,Yessenov University сейчас здесь: Yessenov Uni...,{'data': [{'created_time': '2021-04-13T08:25:5...,{'data': [{'created_time': '2021-04-13T08:25:5...
5,1849247698620787_2867838203428393,2021-04-13T05:52:32+0000,"📌12-ші сәуір күні Yessenov University-де ""Тұлғ...",Yessenov University сейчас здесь: Yessenov Uni...,{'data': [{'created_time': '2021-04-13T07:48:5...,{'data': [{'created_time': '2021-04-13T07:48:5...
6,1849247698620787_2867459396799607,2021-04-12T19:12:44+0000,📌2 сәуір және 9 сәуір күндері Yessenov Univers...,Yessenov University сейчас здесь: Yessenov Uni...,{'data': [{'created_time': '2021-04-13T19:56:2...,{'data': [{'created_time': '2021-04-13T19:56:2...
...,...,...,...,...,...,...
1674,1849247698620787_1963533393858883,2017-10-20T05:17:40+0000,Ш.Есенов атындағы КМТИУ-дың құрметті профессор...,Yessenov University добавил(-а) 16 новых фото ...,{'data': [{'created_time': '2017-10-20T14:54:2...,{'data': [{'created_time': '2017-10-20T14:54:2...
1678,1849247698620787_1963498200529069,2017-10-25T10:11:23+0000,Профессор Б.И. Нұрдаулетованың дәрісі\n\n«Қаза...,Yessenov University добавил(-а) 6 новых фото о...,{'data': [{'created_time': '2017-10-25T16:21:1...,{'data': [{'created_time': '2017-10-25T16:21:1...
1692,1849247698620787_1954940424718180,2017-10-04T14:36:44+0000,«Ұстаздар- ұлағатты тұлға» атты мерекелік шара...,NaN,{'data': [{'created_time': '2017-10-07T14:37:0...,{'data': [{'created_time': '2017-10-07T14:37:0...
1698,1849247698620787_1951614865050736,2017-09-26T11:20:46+0000,Университет оқытушылары мен студенттері Халықа...,NaN,{'data': [{'created_time': '2017-09-29T14:17:0...,{'data': [{'created_time': '2017-09-29T14:17:0...


In [19]:
comment_list = []

for post in df.values.tolist():
    for comment in post[5]['data']:
        comment_list.append(post[:4] + [comment['id'], comment['message'], comment['created_time']])

In [20]:
comment_df = pd.DataFrame(comment_list, 
                          columns=['id', 'created_time', 'post_text', 'story', 
                                   'comment_id', 'comment_text', 'comment_created_time'])
comment_df = comment_df[comment_df.comment_text.apply(lambda x: True if len(x.split(' ')) > 1 else False)]

In [21]:
import re

def cleaning_data(text):
#     if text.split(' ')[0][0] == "@":
#         text = ' '.join(text.split(" ")[1:])
    text = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', text)
    text = str(text).replace('(<br/>)', '')
    text = text.replace('(<a).*(>).*(</a>)', '')
    text = text.replace('(&amp)', '')
    text = text.replace('(&gt)', '')
    text = text.replace('(&lt)', '')
    text = text.replace('(\xa0)', ' ')
    text = text.replace('-', ' ')
    text = text.replace('(', ' ')
    text = text.replace(')', ' ')
    text = re.sub('[^А-Я,а-я,Ә,І,Ң,Ғ,Ү,Ұ,Қ,Ө,Һ,ә,і,ə,ң,ғ,ү,ұ,қ,ө,һ]', ' ', str(text).replace('-', ''))
    text = re.sub('_', '', text)
    text = re.sub('\s+', ' ', text)
    return str(text).lower().strip()

In [22]:
comment_df['post_text'] = comment_df.post_text.astype(str).apply(cleaning_data)
comment_df['comment_text'] = comment_df.comment_text.astype(str).apply(cleaning_data)
comment_df.head()

,id,created_time,post_text,story,comment_id,comment_text,comment_created_time
0,1849247698620787_2868153750063505,2021-04-13T15:51:44+0000,сәуір күні ұйымдастыруымен жемқорлыққа қарсы м...,Yessenov University сейчас здесь: Yessenov Uni...,2868153750063505_2868277350051145,жарайсындар,2021-04-13T18:52:15+0000
1,1849247698620787_2868153750063505,2021-04-13T15:51:44+0000,сәуір күні ұйымдастыруымен жемқорлыққа қарсы м...,Yessenov University сейчас здесь: Yessenov Uni...,2868153750063505_2868155300063350,тағылымды дәріс болды,2021-04-13T15:54:57+0000
2,1849247698620787_2868153750063505,2021-04-13T15:51:44+0000,сәуір күні ұйымдастыруымен жемқорлыққа қарсы м...,Yessenov University сейчас здесь: Yessenov Uni...,2868153750063505_2868620773350136,"рахмет уйымдастырушыларга, кызыкты кездесу болды",2021-04-14T04:04:54+0000
3,1849247698620787_2868153750063505,2021-04-13T15:51:44+0000,сәуір күні ұйымдастыруымен жемқорлыққа қарсы м...,Yessenov University сейчас здесь: Yessenov Uni...,2868153750063505_2868183710060509,оң болсын бастама,2021-04-13T16:52:29+0000
4,1849247698620787_2868153750063505,2021-04-13T15:51:44+0000,сәуір күні ұйымдастыруымен жемқорлыққа қарсы м...,Yessenov University сейчас здесь: Yessenov Uni...,2868153750063505_2868348833377330,жақсы дәріс,2021-04-13T19:55:12+0000


In [23]:
filtired_df = comment_df[comment_df['comment_text'].apply(lambda x: True if len(x.split(' ')) > 1 else False)]
filtired_df

,id,created_time,post_text,story,comment_id,comment_text,comment_created_time
1,1849247698620787_2868153750063505,2021-04-13T15:51:44+0000,сәуір күні ұйымдастыруымен жемқорлыққа қарсы м...,Yessenov University сейчас здесь: Yessenov Uni...,2868153750063505_2868155300063350,тағылымды дәріс болды,2021-04-13T15:54:57+0000
2,1849247698620787_2868153750063505,2021-04-13T15:51:44+0000,сәуір күні ұйымдастыруымен жемқорлыққа қарсы м...,Yessenov University сейчас здесь: Yessenov Uni...,2868153750063505_2868620773350136,"рахмет уйымдастырушыларга, кызыкты кездесу болды",2021-04-14T04:04:54+0000
3,1849247698620787_2868153750063505,2021-04-13T15:51:44+0000,сәуір күні ұйымдастыруымен жемқорлыққа қарсы м...,Yessenov University сейчас здесь: Yessenov Uni...,2868153750063505_2868183710060509,оң болсын бастама,2021-04-13T16:52:29+0000
4,1849247698620787_2868153750063505,2021-04-13T15:51:44+0000,сәуір күні ұйымдастыруымен жемқорлыққа қарсы м...,Yessenov University сейчас здесь: Yessenov Uni...,2868153750063505_2868348833377330,жақсы дәріс,2021-04-13T19:55:12+0000
5,1849247698620787_2868147476730799,2021-04-13T15:39:30+0000,білім беру мектебінің ұйымдастыруымен сәуір кү...,Yessenov University сейчас здесь: Yessenov Uni...,2868147476730799_2868349103377303,жақсы бастама,2021-04-13T19:55:30+0000
...,...,...,...,...,...,...,...
4588,1849247698620787_1963498200529069,2017-10-25T10:11:23+0000,профессор б и нұрдаулетованың дәрісі қазақ тіл...,Yessenov University добавил(-а) 6 новых фото о...,1963498040529085_1963508637194692,"бибайша ильясовна, мықты профессор",2017-10-25T10:41:57+0000
4589,1849247698620787_1954940424718180,2017-10-04T14:36:44+0000,ұстаздар ұлағатты тұлға атты мерекелік шара өт...,NaN,1954940424718180_1956190337926522,спасибо профкому за т плые поздравления и пода...,2017-10-07T14:37:06+0000
4592,1849247698620787_1954940424718180,2017-10-04T14:36:44+0000,ұстаздар ұлағатты тұлға атты мерекелік шара өт...,NaN,1954940424718180_1956628157882740,сембек красавчик,2017-10-08T14:10:34+0000
4593,1849247698620787_1951614865050736,2017-09-26T11:20:46+0000,университет оқытушылары мен студенттері халықа...,NaN,1951614865050736_1952841451594744,берік бақытжанұлы бұл жоо нын жоғары деңгейге ...,2017-09-29T14:17:09+0000


## Data reading

In [24]:
filtired_df = pd.read_csv('translated.txt', sep='\t')
filtired_df.columns = columns=['id', 'created_time', 'post_text', 'story', 
                                   'comment_id', 'comment_text', 'comment_created_time']
filtired_df

,id,created_time,post_text,story,comment_id,comment_text,comment_created_time
1,1849247698620787_2868153750063505,2021-04-13T15: 51: 44 + 0000,Г-н Есенов провел онлайн-видеолекцию по антико...,Университет Есенова теперь здесь: Университет ...,2868153750063505_2868155300063350,была поучительной лекцией,2021-04-13T15: 54: 57 + 0000
2,1849247698620787_2868153750063505,2021-04-13T15: 51: 44 + 0000,Г-н Есенов провел онлайн-видеолекцию по антико...,Университет Есенова теперь здесь: Университет ...,2868153750063505_2868620773350136,"Спасибо организаторам, это была интересная вст...",2021-04-14T04: 04: 54 + 0000
3,1849247698620787_2868153750063505,2021-04-13T15: 51: 44 + 0000,Г-н Есенов провел онлайн-видеолекцию по антико...,Университет Есенова теперь здесь: Университет ...,2868153750063505_2868183710060509,Пусть инициатива будет положительной,2021-04-13T16: 52: 29 + 0000
4,1849247698620787_2868153750063505,2021-04-13T15: 51: 44 + 0000,Г-н Есенов провел онлайн-видеолекцию по антико...,Университет Есенова теперь здесь: Университет ...,2868153750063505_2868348833377330,хорошая лекция,2021-04-13T19: 55: 12 + 0000
5,1849247698620787_2868147476730799,2021-04-13T15: 39: 30 + 0000,Круглый стол на тему «Менеджмент образования и...,Университет Есенова теперь здесь: Университет ...,2868147476730799_2868349103377303,хорошее начало,2021-04-13T19: 55: 30 + 0000
...,...,...,...,...,...,...,...
4588,1849247698620787_1963498200529069,2017-10-25T10: 11: 23 + 0000,"Профессор Б. Нурдаулетова, профессор кафедры к...",Есеновский университет добавил 6 новых фотогра...,1963498040529085_1963508637194692,"Бибайша Ильясовна, сильный профессор",2017-10-25T10: 41: 57 + 0000
4589,1849247698620787_1954940424718180,2017-10-04T14: 36: 44 + 0000,В октябре этого года в университете имени Ш. Е...,NotAddedInfo,1954940424718180_1956190337926522,спасибо профсоюзу за трепые поздравления и под...,2017-10-07T14: 37: 06 + 0000
4592,1849247698620787_1954940424718180,2017-10-04T14: 36: 44 + 0000,В октябре этого года в университете имени Ш. Е...,NotAddedInfo,1954940424718180_1956628157882740,Сембек красив,2017-10-08T14: 10:34 + 0000
4593,1849247698620787_1951614865050736,2017-09-26T11: 20: 46 + 0000,Ректоры и студенты университета приняли участи...,NotAddedInfo,1951614865050736_1952841451594744,"Понятно, что Берик Бакытжанович поднимет этот ...",2017-09-29T14: 17: 09 + 0000


## Sentimental model classifier

In [10]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [11]:
results = pd.DataFrame(model.predict(filtired_df.comment_text.astype(str)))
results['post_id'] = filtired_df['id'].values
results['post_text'] = filtired_df['post_text'].astype(str).apply(cleaning_data).values
results['comment_text'] = filtired_df['comment_text'].astype(str).apply(cleaning_data).values
results['comment_id'] = filtired_df['comment_id'].values
results = results[['post_id', 'post_text', 'comment_text',
                   'comment_id', 'positive', 'negative', 'neutral', 'skip', 'speech', ]]
results['negative_prob'] = (results.negative / results.loc[:, ['negative', 'positive',
                                                             'skip', 'neutral', 'speech',]].sum(axis=1)) * 100
results['positive_prob'] = (results.positive / results.loc[:, ['negative', 'positive',
                                                             'skip', 'neutral', 'speech',]].sum(axis=1)) * 100
results['negative_boolean'] = results.negative_prob > results.positive_prob
results

,post_id,post_text,comment_text,comment_id,positive,negative,neutral,skip,speech,negative_prob,positive_prob,negative_boolean
0,1849247698620787_2868153750063505,г н есенов провел онлайн видеолекцию по антико...,была поучительной лекцией,2868153750063505_2868155300063350,0.160276,0.015435,0.901931,0.003955,0.000789,1.425975,14.807658,False
1,1849247698620787_2868153750063505,г н есенов провел онлайн видеолекцию по антико...,"спасибо организаторам, это была интересная вст...",2868153750063505_2868620773350136,0.206904,0.008857,0.026769,0.014967,0.075868,2.656946,62.065143,False
2,1849247698620787_2868153750063505,г н есенов провел онлайн видеолекцию по антико...,пусть инициатива будет положительной,2868153750063505_2868183710060509,0.019134,0.008587,0.966924,0.005070,0.009718,0.850723,1.895521,False
3,1849247698620787_2868153750063505,г н есенов провел онлайн видеолекцию по антико...,хорошая лекция,2868153750063505_2868348833377330,0.960371,0.001711,0.015916,0.000617,0.000010,0.174809,98.134711,False
4,1849247698620787_2868147476730799,круглый стол на тему менеджмент образования и ...,хорошее начало,2868147476730799_2868349103377303,0.992664,0.003955,0.002811,0.006703,0.000010,0.393126,98.660312,False
...,...,...,...,...,...,...,...,...,...,...,...,...
3950,1849247698620787_1963498200529069,"профессор б нурдаулетова, профессор кафедры ка...","бибайша ильясовна, сильный профессор",1963498040529085_1963508637194692,0.036230,0.031154,0.500010,0.164526,0.000921,4.251101,4.943778,False
3951,1849247698620787_1954940424718180,в октябре этого года в университете имени ш ес...,спасибо профсоюзу за трепые поздравления и под...,1954940424718180_1956190337926522,0.362979,0.024433,0.007356,0.042098,0.798197,1.978288,29.389538,False
3952,1849247698620787_1954940424718180,в октябре этого года в университете имени ш ес...,сембек красив,1954940424718180_1956628157882740,0.991163,0.000010,0.006300,0.003183,0.000010,0.000999,99.050373,False
3953,1849247698620787_1951614865050736,ректоры и студенты университета приняли участи...,"понятно, что берик бакытжанович поднимет этот ...",1951614865050736_1952841451594744,0.080367,0.268951,0.577505,0.106701,0.002561,25.958414,7.756835,True


In [ ]:
from datetime import datetime
filtired_df['post_datetime'] = filtired_df.media_created_at.apply(lambda x: datetime.utcfromtimestamp(x).strftime('%d.%m.%Y %H:%M:%S'))
filtired_df['comment_datetime'] = filtired_df.comment_created_at.apply(lambda x: datetime.utcfromtimestamp(x).strftime('%d.%m.%Y %H:%M:%S'))
filtired_df

In [13]:
results.to_excel('excel/facebook_comments.xlsx')

## Top10 words for posts

In [14]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

import stanza
nlp = stanza.Pipeline(lang='ru', processors = "tokenize,lemma", tokenize_batch_size=16)

def stemming(sentences):
    return ' '.join([stemmer.stem(word) for word in sentences.split()])

def lemmatizing(text):
    return ' '.join([j.lemma for i in nlp(text).sentences for j in i.words])

2021-04-22 13:07:08 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| lemma     | syntagrus |

2021-04-22 13:07:08 INFO: Use device: gpu
2021-04-22 13:07:08 INFO: Loading: tokenize
2021-04-22 13:07:10 INFO: Loading: lemma
2021-04-22 13:07:10 INFO: Done loading processors!


In [15]:
lemmatized = results.drop_duplicates('post_id').post_text.apply(lemmatizing)

In [18]:
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=[lemmatizing(word) for word in stopwords.words('russian')])
X_tfidf = tfidf.fit_transform(lemmatized).toarray()
vocab = tfidf.vocabulary_
reverse_vocab = {v:k for k,v in vocab.items()}

feature_names = tfidf.get_feature_names()
idx = (-X_tfidf).argsort(axis=1)

tfidf_max10 = idx[:, :20]
tfidf_weight = -np.sort(-X_tfidf, axis=1)[:, :20]

df_tfidf = pd.DataFrame([[reverse_vocab.get(item) for item in row] for row in tfidf_max10])
cl_names = ['top_' + str(i+1) for i in range(20)]
df_tfidf.columns = cl_names
df_tfidf['post_id'] = results.drop_duplicates('post_id')['post_id'].values
df_tfidf = df_tfidf.set_index('post_id')
df_tfidf = df_tfidf[cl_names]
df_tfidf[['weight_' + str(i+1) for i in range(20)]] = -np.sort(-X_tfidf, axis=1)[:, :20]
df_tfidf = df_tfidf[np.array([[df_tfidf.columns[i], df_tfidf.columns[i+20]] for i in range(20)]).reshape(20*2)]
df_tfidf

,top_1,weight_1,top_2,weight_2,top_3,weight_3,top_4,weight_4,top_5,weight_5,...,top_16,weight_16,top_17,weight_17,top_18,weight_18,top_19,weight_19,top_20,weight_20
post_id,,,,,,,,,,,,,,,,,,,,,
1849247698620787_2868153750063505,видеолекций,0.571383,антикоррупционный,0.332566,полезный,0.234988,получить,0.228441,ответ,0.216275,...,исчерпывающий,0.121701,узнать,0.121701,химический,0.113937,присутствовать,0.113937,полный,0.108137
1849247698620787_2868147476730799,код,0.256901,гимназия,0.240073,ссылка,0.228134,доступ,0.228134,присоединиться,0.228134,...,онлайн,0.145920,магтрантрань,0.128450,газбильный,0.128450,аятжанный,0.128450,ахметжановичь,0.128450
1849247698620787_2867867713425442,пусть,0.547798,ваш,0.455083,свет,0.401405,лицо,0.258352,сердце,0.237778,...,мир,0.088344,поздравлять,0.078057,принять,0.069926,казойстано,0.067958,предложение,0.000000
1849247698620787_2867838203428393,роль,0.328982,аллабергенный,0.246977,конарбаи,0.246977,президиум,0.230799,семейный,0.203144,...,получить,0.131656,вопрос,0.126467,вторжение,0.123488,штаб,0.123488,встреча,0.119355
1849247698620787_2863965843815629,ветеран,0.503161,нефтяник,0.243136,встреча,0.195832,регион,0.182360,труд,0.163995,...,любимый,0.133323,профессия,0.129287,случай,0.122706,ответ,0.122706,открытый,0.117452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1849247698620787_1963011817244374,соревнование,0.576152,команда,0.238988,менеджмент,0.202276,лучший,0.187117,место,0.167022,...,факультет,0.115696,мендалие,0.108725,физкультура,0.108725,наурызбый,0.108725,судья,0.101604
1849247698620787_1963533393858883,поэт,0.488105,эсенгальный,0.306257,профессор,0.259042,литература,0.247884,читать,0.201523,...,затем,0.104369,писать,0.104369,лауреат,0.100761,студент,0.096064,кгот,0.095068
1849247698620787_1963498200529069,жырать,0.282157,наука,0.227470,республика,0.196450,язык,0.194238,век,0.192313,...,бибой,0.144235,лекция,0.128160,право,0.118356,терминология,0.112863,мозговой,0.112863


In [20]:
results.groupby('post_id').mean().iloc[:,:7]

,positive,negative,neutral,skip,speech,negative_prob,positive_prob
post_id,,,,,,,
1849247698620787_1085067278520596,0.013876,0.167305,0.849451,0.010492,0.000010,15.692532,1.354609
1849247698620787_1951614865050736,0.041315,0.167283,0.492424,0.077784,0.266890,16.088009,3.985583
1849247698620787_1954940424718180,0.677071,0.012222,0.006828,0.022640,0.399103,0.989644,64.219956
1849247698620787_1963011817244374,0.065615,0.010338,0.004209,0.006108,0.968866,0.979794,6.218617
1849247698620787_1963498200529069,0.064464,0.111548,0.345976,0.175835,0.006681,16.318981,9.326533
...,...,...,...,...,...,...,...
2031298790415676_2031370527075169,0.000010,0.000010,1.000010,0.000010,0.000010,0.001000,0.001000
2037357129809842_2037370836475138,0.000010,0.000010,1.000010,0.000010,0.000010,0.001000,0.001000
2041496359395919_2042129615999260,0.000010,0.000010,1.000010,0.000010,0.000010,0.001000,0.001000


## Clustering

In [22]:
posts_dataframe = results.drop_duplicates('post_id')[['post_id', 'post_text']]
posts_dataframe = posts_dataframe.set_index('post_id')
posts_dataframe

,post_text
post_id,
1849247698620787_2868153750063505,г н есенов провел онлайн видеолекцию по антико...
1849247698620787_2868147476730799,круглый стол на тему менеджмент образования и ...
1849247698620787_2867867713425442,поздравляет всех вас с наступлением священного...
1849247698620787_2867838203428393,апреля в есеновском университете прошла встреч...
1849247698620787_2863965843815629,в апреле в формате открытого диалога прошла вс...
...,...
1849247698620787_1963011817244374,в октябре этого года кафедрой менеджмента экон...
1849247698620787_1963533393858883,встреча с почетным профессором кыргызского гос...
1849247698620787_1963498200529069,"профессор б нурдаулетова, профессор кафедры ка..."


In [23]:
train_df = pd.concat([posts_dataframe, results.groupby('post_id').mean().loc[:,['negative', 'positive',
                                                             'skip', 'neutral', 'speech',]]], axis=1)
train_df

,post_text,negative,positive,skip,neutral,speech
1849247698620787_2868153750063505,г н есенов провел онлайн видеолекцию по антико...,0.008648,0.336671,0.006152,0.477885,0.021597
1849247698620787_2868147476730799,круглый стол на тему менеджмент образования и ...,0.014572,0.503589,0.004993,0.498545,0.000010
1849247698620787_2867867713425442,поздравляет всех вас с наступлением священного...,0.007664,0.010051,0.014149,0.935019,0.001518
1849247698620787_2867838203428393,апреля в есеновском университете прошла встреч...,0.011697,0.000468,0.056662,0.012831,0.938134
1849247698620787_2863965843815629,в апреле в формате открытого диалога прошла вс...,0.056662,0.196836,0.008072,0.500010,0.082707
...,...,...,...,...,...,...
1849247698620787_1963011817244374,в октябре этого года кафедрой менеджмента экон...,0.010338,0.065615,0.006108,0.004209,0.968866
1849247698620787_1963533393858883,встреча с почетным профессором кыргызского гос...,0.233716,0.067557,0.206904,0.281416,0.008587
1849247698620787_1963498200529069,"профессор б нурдаулетова, профессор кафедры ка...",0.111548,0.064464,0.175835,0.345976,0.006681
1849247698620787_1954940424718180,в октябре этого года в университете имени ш ес...,0.012222,0.677071,0.022640,0.006828,0.399103


In [24]:
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_clus = TfidfVectorizer(stop_words=[lemmatizing(word) for word in stopwords.words('russian')], max_df=0.1, min_df=0.001, )
X_tfidf_clus = tfidf_clus.fit_transform(results.drop_duplicates('post_id').post_text.apply(lemmatizing)).toarray()
print(X_tfidf_clus.shape)

(358, 4731)


In [25]:
from sklearn.cluster import KMeans
from collections import Counter
cluster_count = 17
model = KMeans(n_clusters=cluster_count, random_state=0).fit(np.hstack([X_tfidf_clus, train_df.loc[:, ['negative', 'positive',
                                                             'skip', 'neutral', 'speech',]].values]))
model_prediction = model.predict(np.hstack([X_tfidf_clus, train_df.loc[:, ['negative', 'positive',
                                                             'skip', 'neutral', 'speech',]].values]))
print(Counter(model_prediction).most_common())

[(2, 48), (10, 34), (15, 31), (4, 26), (9, 26), (14, 20), (11, 20), (0, 19), (12, 19), (1, 18), (5, 16), (7, 16), (16, 16), (8, 13), (3, 13), (13, 12), (6, 11)]


In [15]:
train_df['cluster_id'] = model_prediction
# train_df.to_excel('cluster_posts.xlsx')
train_df

NameError: name 'model_prediction' is not defined

In [27]:
cluster_text = {}
for i in range(cluster_count):
    cluster_text[i] = ' '.join(train_df[train_df.cluster_id == i].post_text.values)

In [28]:
cluster_text.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])

In [29]:
tfidf_clus_top = TfidfVectorizer(stop_words=[lemmatizing(word) for word in stopwords.words('russian')])
tfidf_clus_top_arr = tfidf_clus_top.fit_transform([lemmatizing(clusses) for clusses in list(cluster_text.values())]).toarray()
vocab = tfidf_clus_top.vocabulary_
reverse_vocab = {v:k for k,v in vocab.items()}

feature_names = tfidf_clus_top.get_feature_names()

In [30]:
(-np.sort(-tfidf_clus_top_arr, axis=1)[:, :100]).shape

(17, 100)

In [34]:
pd.DataFrame([[reverse_vocab.get(item) for item in row] for row in tfidf_max10])

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,наука,университет,лучший,педагогика,кафедра,поздравлять,преподаватель,образование,желать,алмажый,...,изобретательный,таубаевич,поздравлятый,иманбай,радиотехник,жоламан,металлургический,батыр,разряд,электроника
1,поэт,университет,айбергенов,вечер,наука,талант,казойстано,студент,есеновский,лауреат,...,стать,область,час,организатор,внести,конец,практический,работа,молодой,бижатый
2,университет,есеновский,колледж,студент,менеджмент,кафедра,образование,образовательный,актой,место,...,язык,молодежь,приемная,фильм,партия,назарбаево,статья,работать,организация,морской
3,университет,мангавыставский,есеновский,студент,участие,непрерывный,встреча,мероприятие,казойстано,партия,...,токай,краевед,миржакый,знакомый,поддержать,блюдо,жомартый,айгуль,ассамблея,предпринимательство
4,поздравлять,творческий,университет,квесто,здоровье,крепкий,желать,наука,долголетие,молодой,...,экскурсия,жантор,аллабергенный,жанториный,ержанов,актер,кузембаевный,риск,конарбаи,пажарбекович
5,поэт,университет,студент,мукагать,макатьев,конкурс,профессор,литература,встреча,республика,...,первокурсник,принадлежность,свободно,второй,также,премия,делать,наградить,курс,бизнес
6,конференция,университет,научный,прикаспийский,кондыбый,международный,профессор,есеновский,чоудхурий,дабабратый,...,мадий,двойной,коммунистический,чжан,жузберген,океанология,эркебуланный,ауэз,протез,караногай
7,университет,программа,франций,республика,областной,иран,исламский,есеновский,учебный,мангистауский,...,жума,аннагельдиевич,оринбаевич,сапармурат,филиппый,рост,бриджий,показ,универсал,огулгозель
8,неделя,кондыбый,университет,культура,мифология,программа,будущий,юрист,область,азербайджанный,...,возрождение,специалист,различный,апрель,день,который,анализ,налоговый,увеличить,азербайджанский
9,университет,меморандум,сотрудничество,студент,подписать,есеновский,промышленный,журнал,развитие,технология,...,ректор,программный,продвижение,шолпан,конкурентоспособность,визит,бизнес,учебный,профессор,республика


In [36]:
idx = (-tfidf_clus_top_arr).argsort(axis=1)

tfidf_max10 = idx[:, :100]
tfidf_weight = -np.sort(-tfidf_clus_top_arr, axis=1)[:, :100]

df_tfidf_c = pd.DataFrame([[reverse_vocab.get(item) for item in row] for row in tfidf_max10])
df_tfidf_c = df_tfidf_c.T
df_tfidf_c.index = ['top_' + str(i+1) for i in range(100)]
df_tfidf_c.columns = ['cluster_' + str(i+1) for i in range(cluster_count)]
df_tfidf_c = df_tfidf_c.T
df_tfidf_c[['weight_' + str(i+1) for i in range(100)]] = tfidf_weight
df_tfidf_c = df_tfidf_c[np.array([[df_tfidf_c.columns[i], df_tfidf_c.columns[i+100]] for i in range(100)]).reshape(100*2)]
df_tfidf_c

,top_1,weight_1,top_2,weight_2,top_3,weight_3,top_4,weight_4,top_5,weight_5,...,top_96,weight_96,top_97,weight_97,top_98,weight_98,top_99,weight_99,top_100,weight_100
cluster_1,наука,0.217851,университет,0.198440,лучший,0.170866,педагогика,0.163248,кафедра,0.162094,...,жоламан,0.048804,металлургический,0.048804,батыр,0.048804,разряд,0.048804,электроника,0.048804
cluster_2,поэт,0.284566,университет,0.215723,айбергенов,0.197062,вечер,0.154644,наука,0.143348,...,конец,0.041743,практический,0.041743,работа,0.041337,молодой,0.040846,бижатый,0.039412
cluster_3,университет,0.352678,есеновский,0.156746,колледж,0.145245,студент,0.132254,менеджмент,0.130804,...,назарбаево,0.044691,статья,0.044352,работать,0.043864,организация,0.043802,морской,0.041468
cluster_4,университет,0.366392,мангавыставский,0.150184,есеновский,0.140920,студент,0.122131,участие,0.122131,...,блюдо,0.052455,жомартый,0.052455,айгуль,0.052455,ассамблея,0.051039,предпринимательство,0.051039
cluster_5,поздравлять,0.227659,творческий,0.218179,университет,0.192375,квесто,0.175733,здоровье,0.174493,...,актер,0.043933,кузембаевный,0.043933,риск,0.043933,конарбаи,0.043933,пажарбекович,0.043933
cluster_6,поэт,0.281320,университет,0.275548,студент,0.223417,мукагать,0.145536,макатьев,0.142863,...,премия,0.046887,делать,0.046887,наградить,0.046594,курс,0.046594,бизнес,0.046594
cluster_7,конференция,0.407242,университет,0.253060,научный,0.170918,прикаспийский,0.157323,кондыбый,0.156996,...,океанология,0.044949,эркебуланный,0.044949,ауэз,0.044949,протез,0.044949,караногай,0.044949
cluster_8,университет,0.269076,программа,0.189638,франций,0.167280,республика,0.156961,областной,0.156174,...,рост,0.047794,бриджий,0.047794,показ,0.047794,универсал,0.047794,огулгозель,0.047794
cluster_9,неделя,0.213623,кондыбый,0.211702,университет,0.189578,культура,0.186512,мифология,0.181837,...,который,0.050103,анализ,0.047472,налоговый,0.047472,увеличить,0.047472,азербайджанский,0.047472
cluster_10,университет,0.393698,меморандум,0.176475,сотрудничество,0.161304,студент,0.155920,подписать,0.155298,...,визит,0.039044,бизнес,0.039021,учебный,0.039021,профессор,0.039021,республика,0.038980


In [99]:
df_tfidf_c.to_excel('excel/facebook_cluster_top100weights.xlsx')

In [13]:
last_otchet = results.drop_duplicates('post_id')[['post_id', 'post_text', 'negative_prob', 'positive_prob']]
last_otchet = last_otchet.set_index('post_id')
last_otchet

,post_text,negative_prob,positive_prob
post_id,,,
1849247698620787_2868153750063505,г н есенов провел онлайн видеолекцию по антико...,1.425975,14.807658
1849247698620787_2868147476730799,круглый стол на тему менеджмент образования и ...,0.393126,98.660312
1849247698620787_2867867713425442,поздравляет всех вас с наступлением священного...,1.947411,1.118466
1849247698620787_2867838203428393,апреля в есеновском университете прошла встреч...,1.147022,0.045922
1849247698620787_2863965843815629,в апреле в формате открытого диалога прошла вс...,6.711268,23.313871
...,...,...,...
1849247698620787_1963011817244374,в октябре этого года кафедрой менеджмента экон...,0.979794,6.218617
1849247698620787_1963533393858883,встреча с почетным профессором кыргызского гос...,29.281153,8.463839
1849247698620787_1963498200529069,"профессор б нурдаулетова, профессор кафедры ка...",28.386861,13.709288


In [14]:
last_otchet['cluster_label'] = train_df.cluster_id
last_otchet['top20keywords'] = df_tfidf.T.apply(lambda x: ', '.join(x))
last_otchet['cluster_top100keywords'] = last_otchet.cluster_label.apply(lambda x: df_tfidf_c.T.apply(lambda x: ', '.join(x))[x])
last_otchet

NameError: name 'train_df' is not defined

In [31]:
pd.read_excel('excel/facebook_dates_likes_comments.xlsx')

,post_id,datetime,likes,comments,shares
0,1849247698620787_1085067278520596,NaN,0,0,None
1,1849247698620787_1951614865050736,26.09.2017,16,2,None
2,1849247698620787_1954940424718180,04.10.2017,34,4,4
3,1849247698620787_1963011817244374,NaN,0,0,None
4,1849247698620787_1963498200529069,NaN,0,0,None
...,...,...,...,...,...
353,2031298790415676_2031370527075169,NaN,0,0,None
354,2037357129809842_2037370836475138,NaN,0,0,None
355,2041496359395919_2042129615999260,NaN,0,0,None
356,2045791372299751_2046020808943474,NaN,0,0,None


In [39]:
last_otchet = pd.read_excel('excel/facebook_media.xlsx')
last_otchet['Datetime'] = pd.read_excel('excel/facebook_dates_likes_comments.xlsx').datetime
last_otchet['likes'] = pd.read_excel('excel/facebook_dates_likes_comments.xlsx').likes
last_otchet['comments'] = pd.read_excel('excel/facebook_dates_likes_comments.xlsx').comments
last_otchet['shares'] = pd.read_excel('excel/facebook_dates_likes_comments.xlsx').shares
last_otchet

,Unnamed: 0,post_id,post_text,negative_prob,positive_prob,cluster_label,top20keywords,cluster_top100keywords,Datetime,likes,comments,shares
0,0,1849247698620787_2868153750063505,г н есенов провел онлайн видеолекцию по антико...,1.425975,14.807658,8,"полный, химический, присутствовать, узнать, ис...","увеличить, персидский, исполнение, налоговый, ...",NaN,0,0,None
1,1,1849247698620787_2868147476730799,круглый стол на тему менеджмент образования и ...,0.393126,98.660312,14,"ахметжанович, ахметжановичь, аятжанный, аятжан...","являються, действенный, беркут, самооценкий, о...",26.09.2017,16,2,None
2,2,1849247698620787_2867867713425442,поздравляет всех вас с наступлением священного...,1.947411,1.118466,2,"караджанба, казойстано, принять, поздравлять, ...","занять, организация, работать, статья, приемна...",04.10.2017,34,4,4
3,3,1849247698620787_2867838203428393,апреля в есеновском университете прошла встреч...,1.147022,0.045922,4,"встреча, штаб, вторжение, вопрос, получить, ор...","конарбаи, кузембаевный, аллабергенный, солнечн...",NaN,0,0,None
4,4,1849247698620787_2863965843815629,в апреле в формате открытого диалога прошла вс...,6.711268,23.313871,1,"открытый, ответ, случай, профессия, организато...","перевод, молодой, работа, практический, конец,...",NaN,0,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...
353,353,1849247698620787_1963011817244374,в октябре этого года кафедрой менеджмента экон...,0.979794,6.218617,11,"юп, мендалие, наурызбый, физкультура, факульте...","звание, журнал, пройти, страна, год, март, каз...",NaN,0,0,None
354,354,1849247698620787_1963533393858883,встреча с почетным профессором кыргызского гос...,29.281153,8.463839,5,"кгот, студент, лауреат, затем, писать, совреме...","курс, наградить, бизнес, делать, премия, также...",NaN,0,0,None
355,355,1849247698620787_1963498200529069,"профессор б нурдаулетова, профессор кафедры ка...",28.386861,13.709288,5,"мозговой, концептуальный, право, лекция, бибой...","курс, наградить, бизнес, делать, премия, также...",NaN,0,0,None
356,356,1849247698620787_1954940424718180,в октябре этого года в университете имени ш ес...,1.978288,29.389538,3,"добившийся, профсоюз, смысл, посвятить, студен...","клуб, предпринимательство, айгуль, короткий, м...",NaN,0,0,None


In [40]:
last_otchet.to_excel('excel/facebook_media.xlsx')